In [1]:

from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from src.data_utils import load_and_clean_data

data_dir = Path('./data/')
raw_data_path = data_dir / 'tweets.txt'

clean_df = load_and_clean_data(raw_data_path)


train_df, temp_df = train_test_split(clean_df, test_size=0.2, random_state=42)

val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


print("4. Сохранение файлов...")
train_path = data_dir / 'train.csv'
val_path = data_dir / 'val.csv'
test_path = data_dir / 'test.csv'

train_df.to_csv(train_path, index=False)
val_df.to_csv(val_path, index=False)
test_df.to_csv(test_path, index=False)

print("\nПроцесс завершен! Файлы сохранены:")
print(f"  - Обучающая выборка: {train_path} ({len(train_df)} строк)")
print(f"  - Валидационная выборка: {val_path} ({len(val_df)} строк)")
print(f"  - Тестовая выборка: {test_path} ({len(test_df)} строк)")


print("\n--- Пример данных из созданного train.csv ---")
print(train_df.head())

1. Чтение сырого файла: data\tweets.txt
2. Очистка текста...
Готово. Получено 1596801 чистых строк.
4. Сохранение файлов...

Процесс завершен! Файлы сохранены:
  - Обучающая выборка: data\train.csv (1277440 строк)
  - Валидационная выборка: data\val.csv (159680 строк)
  - Тестовая выборка: data\test.csv (159681 строк)

--- Пример данных из созданного train.csv ---
                                                     text
86449      or you could be like me work six days in a row
989848                 good morning have a wonderful week
869848  hope this works or a simple pic what a capacit...
938814  just read the sweet message my best friend sen...
75206   why is no one shopping am on my lunch isnt out...


In [2]:
from pathlib import Path
from torch.utils.data import DataLoader
from src.next_token_dataset import TextDataset, PadCollate

data_dir = Path('./data/')
BATCH_SIZE = 128

train_dataset = TextDataset(file_path=data_dir / 'train.csv')

vocab = train_dataset.word2idx
pad_idx = vocab[train_dataset.pad_token]
vocab_size = train_dataset.vocab_size

val_dataset = TextDataset(file_path=data_dir / 'val.csv', vocab=vocab)
test_dataset = TextDataset(file_path=data_dir / 'test.csv', vocab=vocab)

collate_fn = PadCollate(pad_idx=pad_idx)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

print(f"Размер словаря: {vocab_size}")
print("DataLoader'ы готовы.")

inputs, targets = next(iter(train_loader))
print(f"Размер батча (вход): {inputs.shape}")
print(f"Размер батча (цель): {targets.shape}")

Размер словаря: 20000
DataLoader'ы готовы.
Размер батча (вход): torch.Size([128, 26])
Размер батча (цель): torch.Size([128, 26])


In [ ]:
import torch
from src.lstm_model import LSTMModel

EMBEDDING_DIM = 128
HIDDEN_DIM = 256
NUM_LAYERS = 2
DROPOUT = 0.3

device = "cuda" if torch.cuda.is_available() else "cpu"

model = LSTMModel(
    vocab_size=vocab_size,
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    num_layers=NUM_LAYERS,
    dropout_prob=DROPOUT
).to(device)


inputs, targets = next(iter(train_loader))
inputs, targets = inputs.to(device), targets.to(device)

output = model(inputs)

print("Модель создана и успешно обработала один батч данных.")
print(f"Размер входа: {inputs.shape}")
print(f"Размер выхода (логитов): {output.shape}")
print(f"Ожидаемый размер выхода: ({BATCH_SIZE}, {inputs.shape[1]}, {vocab_size})")


idx2word = train_dataset.idx2word



Модель создана и успешно обработала один батч данных.
Размер входа: torch.Size([128, 26])
Размер выхода (логитов): torch.Size([128, 26, 20000])
Ожидаемый размер выхода: (128, 26, 20000)


In [ ]:
import torch.optim as optim
import torch.nn as nn
from src.lstm_train import train_loop

LEARNING_RATE = 0.001
NUM_EPOCHS = 5 

criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

train_loop(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    num_epochs=NUM_EPOCHS,
    device=device,
    vocab=vocab,
    idx2word=idx2word
)

In [ ]:
from pathlib import Path


model_path = Path('./models/best_lstm_model.pth')
model.load_state_dict(torch.load(model_path, map_location=device))


test_prompts = [
    "i feel so",
    "today is a",
    "i want to",
    "what are you",
    "this is the"
]

print("\n--- Примеры генерации текста ---")
for prompt in test_prompts:
    generated_text = model.generate(
        start_seq=prompt,
        max_len=15,
        vocab=vocab,
        idx2word=idx2word,
        device=device
    )
    print(f"PROMPT: '{prompt}'")
    print(f"GENERATED: '{generated_text}'\n")